In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

from pathlib import Path

if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

In [2]:
spark = SparkSession.builder \
    .appName("assertion-status")\
    .master("local[2]")\
    .config("spark.driver.memory","4G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .getOrCreate()

In [3]:
  documentAssembler = DocumentAssembler()\
      .setInputCol("region")\
      .setMetadataCol("metadata")\
      .setOutputCol("text")

  tokenizer = Tokenizer()\
    .setInputCols(["text"])\
    .setOutputCol("token")

  ocrspellModel = ContextSpellCheckerModel().pretrained()\
    .setInputCols("token")\
    .setOutputCol("spell_checked")


com.johnsnowlabs.nlp.DocumentAssembler


Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadModel.
: java.lang.NoSuchMethodError: org.apache.http.HttpHost.getAddress()Ljava/net/InetAddress;
	at org.apache.http.conn.routing.HttpRoute.normalize(HttpRoute.java:107)
	at org.apache.http.conn.routing.HttpRoute.<init>(HttpRoute.java:75)
	at org.apache.http.conn.routing.HttpRoute.<init>(HttpRoute.java:171)
	at org.apache.http.impl.conn.DefaultHttpRoutePlanner.determineRoute(DefaultHttpRoutePlanner.java:118)
	at org.apache.http.impl.client.DefaultRequestDirector.determineRoute(DefaultRequestDirector.java:761)
	at org.apache.http.impl.client.DefaultRequestDirector.execute(DefaultRequestDirector.java:380)
	at org.apache.http.impl.client.AbstractHttpClient.doExecute(AbstractHttpClient.java:835)
	at org.apache.http.impl.client.CloseableHttpClient.execute(CloseableHttpClient.java:83)
	at org.apache.http.impl.client.CloseableHttpClient.execute(CloseableHttpClient.java:56)
	at co